In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time_templates.utilities.fitting import plot_fit_curve, fit_xy_xerr_yerr
from time_templates.utilities.misc import histedges_equalN
import time_templates.utilities.plot as ttplt
from time_templates.MPD.Xmumax_50 import Xmumax_50_from_Xmax
from time_templates.datareader.get_data import fetch_MC_data_from_tree, fetch_data
from matplotlib.lines import Line2D
import scipy.stats as scistats
from time_templates.preprocessing.apply_cuts_df import apply_cuts_df
from time_templates.utilities.fitting import plot_fit_xy_xerr_yerr
from time_templates.misc.Xmax import Fd_Xmax_resolution_at_lgE
from time_templates import data_path

primary_handles = [Line2D([0], [0], color="#1D5287", lw=10),
                  Line2D([0], [0], color="#BC313D", lw=10)]
primary_labels = ['proton', 'iron']
PRIMARY_COLORS = {'proton': "b", 'iron': "r"}

HIM_LS = {'EPOS_LHC': '-', 'QGSJET-II.04': '--', 'EPOSLHC': '-', 'QGSII04': '--', 'SIB23': ':', 'EPOS-LHC': '-'}
HIM_MARKERS = {'EPOS_LHC': 'o', 'QGSJET-II.04': 'v', 'EPOSLHC': 'o', 'QGSII04': 'v', 'SIB23': 'x', 'EPOS-LHC': 'o'}
from time_templates.utilities.atmosphere import Atmosphere

#Chang here to save figs
SAVE = False

plot_path = '/home/mart/auger/thesis/thesis_overleaf/figs/chapter6/'

atm = Atmosphere()

if SAVE:
    try:
        plt.style.use('thesis')
    except:
        pass
    nboot = 10000
else:
    nboot = 100
    %matplotlib notebook
def savefig(name):
    if SAVE:
        plt.savefig(plot_path+name, bbox_inches='tight')

In [ ]:
# datapath = "/home/mart/auger/data/time_templates/fitted_csv/"


def load_fitted_df(primary, energy, HIM='EPOSLHC', key_lib='rcola_Offlinev3r3p4', kdatapath="/home/mart/auger/data/time_templates/fitted_csv/"):
    df = fetch_MC_data_from_tree(primary=primary, energy=energy, HIM=HIM, key=key_lib)
    df.reset_index(inplace=True)
    df['EventId'] = df['EventId'].astype('str') + primary + energy
    df.set_index('EventId', inplace=True)
    df = df.groupby('EventId').mean()
    df['HIM'] = HIM
    df['primary'] = primary
    return df

def load_dfs():
    dfs = []
    for primary in ['proton', 'iron']:
        for energy in ['19.0_19.5', '19.5_20.0']:
            df = load_fitted_df(primary, energy, HIM='EPOSLHC', key_lib='MCTask_Offline_v3r99p2a')
            df['UUB'] = 'UB v3r99p2a'
            df['HIM'] = 'EPOS-LHC'
            dfs.append(df)
            
    for HIM in ['EPOSLHC', 'QGSII04', 'SIB23']:
        for primary in ['proton', 'iron']:
            for energy in ['19_19.5', '19.5_20']:
                df = load_fitted_df(primary, energy, HIM=HIM, key_lib='rcola_Offlinev3r3p4')
                df['UUB'] = 'UB v3r3p4'
                if HIM == 'EPOSLHC':
                    df['HIM'] = 'EPOS-LHC'
                if HIM == 'QGSII04':
                    df['HIM'] = 'QGSJET-II.04'
                dfs.append(df)
                
    for HIM in ['EPOS_LHC', 'QGSJET-II.04']:
        for primary in ['proton', 'iron']:
            for energy in ['19_19.5', '19.5_20']:
                df = load_fitted_df(primary, energy, HIM=HIM, key_lib='new_UUB_SSD_rcola')
                df['UUB'] = 'UUB trunk'
                if HIM == 'EPOS_LHC':
                    df['HIM'] = 'EPOS-LHC'
                dfs.append(df)
                

    df = pd.concat(dfs)
    df['sigma_Rc_Rc'] = df['Rc_fit_err']/df['Rc_fit']
    df["MCDXmax"] = (
        atm.slant_depth_at_height(1400, df["MCTheta"].values) - df["MCXmaxGH"].values
    )

    return df

df = load_dfs()

In [ ]:
cuts = {'Rc_fit_success': True, 'SdCosTheta_new': (0.6, 1), 'Sdr_new_min': (500, 1000)}
df = apply_cuts_df(df, cuts)
from time_templates.starttime.start_time_deMauro import DX_at_Rc, var_DXRc
df['DXRc'] = DX_at_Rc(df['Rc_fit'].values, np.arccos(df['SdCosTheta_new'].values))
df['DXRc_err'] = np.sqrt(var_DXRc(df['Rc_fit_err'].values, df['Rc_fit'].values, np.arccos(df['SdCosTheta_new'].values)))

In [ ]:
d_fit_p = {}
d_fit_perr = {}
d_fit_chi2 = {}
d_fit_ndf = {}
sigC = 42
from uncertainties import ufloat 

DXRc_func = lambda DXRc, p0, p1, p2, lgE: p0 + p1 * (DXRc-855.4 + p2*(lgE-19.15))

for (UUB, HIM, primary), df_ in df.groupby(['UUB', 'HIM', 'primary']):
    p, perr, chi2, ndf = fit_xy_xerr_yerr(df_['DXRc'], df_['MCDXmax'], xerr=np.sqrt(df_['DXRc_err']**2 + sigC**2), yerr=2.5, 
          func=lambda p, x: DXRc_func(x, *p, df_['MClgE']), p0=[500, 1, -134])
    key = f"{UUB} {HIM} {primary}"
    d_fit_p[key] = p
    d_fit_perr[key] = perr
    d_fit_chi2[key] = chi2
    d_fit_ndf[key] = ndf
    s = [f"${{{ufloat(p[i], perr[i]):.1uL}}}$" for i in range(len(p))]
    s.insert(0, key)
    s.append(f'${chi2:.0f}/{ndf:.0f}$')
#     s.append(' \\\\')
    print(' & '.join(s), '\\\\')

#mix p/Fe does not change fit parameters
d_fit_p

In [ ]:
df_fit = pd.read_csv("/home/mart/auger/data/time_templates/fitted/df_events_fitted_observer_icrc19_Golden_SdlgE19_theta53_selected.csv")
df_data = fetch_data(key='observer_icrc19_Golden_SdlgE19*', get_dates=True, force=False, do_Rc_fit=True)
df_fit.set_index('EventId', inplace=True)
df_data = df_data.groupby('EventId').mean()
df_data = df_fit.join(df_data, on='EventId', rsuffix='r_')
# df_data['p_val_Rc_fit']  = scistats.chi2.sf(df_data['DXmax_Rc_chi2'], df_data['DXmax_Rc_ndof'])

df_data['FdDXmax'] = df_data['Xg'] - df_data['FdXmax']
df_data['FdXmax_err'] = np.maximum(df_data['FdXmax_err'], Fd_Xmax_resolution_at_lgE(df_data['FdlgE']))
df_data['FdDXmax_err'] = df_data['FdXmax_err']

df_data = apply_cuts_df(df_data, cuts)


In [ ]:
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
f, ax = plt.subplots(1, figsize=(4, 3))
x = np.linspace(450, 1100, 12)

for (UUB, HIM, primary), df_ in df.groupby(['UUB', 'HIM', 'primary']):
#     if HIM != 'EPOSLHC':
#         continue
    if 'SIB' in HIM:
        continue
    if 'UUB' in UUB:
        marker = 'v'
    elif 'v3r3p4' in UUB:
        marker = 'x'


#     if primary == 'proton':
#         ax = ax1
#     else:
#         ax = ax2
    if 'vr99' in UUB:
        label = f"\\textbf{{{UUB} {HIM} {primary}}}"
        marker = 'o'
        ms = 5
        alpha = 1
        zorder = 98
    else:
        label = f"{UUB} {HIM} {primary}"
        ms = 2
        alpha = 0.8
        zorder = 1
    ax.plot(x, DXRc_func(x, *d_fit_p[f"{UUB} {HIM} {primary}"], 19.15), color=PRIMARY_COLORS[primary], ls=HIM_LS[HIM], label=label, marker=marker, ms=ms, alpha=alpha,
            zorder=zorder)
    
(a, b, c), perr, chi2, ndf = fit_xy_xerr_yerr(df_data['DXRc_fit'], df_data['FdDXmax'], xerr=np.sqrt(df_data['DXRc_fit_err']**2 + 42**2), yerr=df_data['FdDXmax_err'],
      func=lambda p, x: DXRc_func(x, *p, df_data['SdlgE']), p0=[500, 1, -134])
print(a, b, c, perr, chi2, ndf)
# for ax in [ax1, ax2]:
ax, (p, perr, chi2, ndf) = plot_fit_xy_xerr_yerr(df_data['DXRc_fit']+c*(df_data['SdlgE']-19.15), df_data['FdDXmax'], xerr=np.sqrt(df_data['DXRc_fit_err']**2 + sigC**2), yerr=df_data['FdDXmax_err'],
      func=lambda p, x: DXRc_func(x, p[0], p[1], c, 19.15), p0=[500, 1], ax=ax, ebar_kws=dict(color='k', ls='', alpha=0.2, label='data'), line_kws=dict(lw=2., zorder=99), add_label=False)
ax.legend()
ax.set_xlim([400, 1250])
# ax.set_ylim([100, 600])
lgEeV = "\\log_{10}{\\left(E/\\rm eV\\right)}"

ax.set_xlabel(f'$DX_{{R_c}} - c \\left({lgEeV} - \\overline{{{lgEeV}}}\\right)\, [\\rm g/cm^2]$')
ax.set_ylabel('$DX_{\\mathrm{max}}\, [\\rm g/cm^2]$')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig(plot_path + "correlation_DXRc_DXmax.pdf", bbox_inches='tight')